In [2]:
!pip install Medpy

     |████████████████████████████████| 153kB 2.9MB/s 
     |████████████████████████████████| 42.5MB 72kB/s 
  Created wheel for Medpy: filename=MedPy-0.4.0-cp36-cp36m-linux_x86_64.whl size=753441 sha256=51720e83c524afb27f28cff804989066ec6a5b332e6fd7ef56c03c80806085d2
  Stored in directory: /root/.cache/pip/wheels/8c/c9/9c/2c6281c7a72b9fb1570862a4f028af7ce38405008354fbf870
Successfully built Medpy


In [3]:
import copy
import os
import numpy as np
from medpy.io import load, save
import nibabel
from keras import backend as K
import keras
import tensorflow as tf
from keras.layers import Add, Concatenate, Multiply, Reshape, Dense
from keras.layers import Input
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.convolutional import UpSampling2D
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.optimizers import Adam, SGD
from skimage.transform import resize
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import recall_score, precision_score
import cv2
from tensorflow.keras.metrics import Precision, Recall
from keras.layers.pooling import GlobalAveragePooling2D, MaxPooling2D
from keras.regularizers import l2

image_rows = 256
image_cols = 256
smooth = 1.
weight_decay = 1e-2
# Download the weight and adjust the path accordingly
weight_path = "/content/drive/My Drive/3DIRCAD-Result/weight/weights-resunet++.h5"

K.set_image_data_format('channels_last')

In [ ]:
#-----Create folders-----
# Original test folder, this is where original 3DIRCAD dataset is put.
testing_folder = '/content/drive/My Drive/3DIRCAD/Data'
# After 3DIRCAD dataset is processed, it is saved here.
processed_testing_folder = '/content/drive/My Drive/3DIRCAD-Preprocessed-test'
# This is where 3DIRCAD dataset is transformed into numpy array file saved. 
numpy_folder = '/content/drive/My Drive/3DIRCAD-Preprocessed-test/numpy'

if not os.path.exists(testing_folder):
  print('Creat test folder')
  os.mkdir(testing_folder)
else:
  print('Test folder exists')

if not os.path.exists(processed_testing_folder):
  print('Creat processed test folder')
  os.mkdir(processed_testing_folder)
else:
  print('Processed test folder exists')

if not os.path.exists(numpy_folder):
  print('Creat numpy array folder')
  os.mkdir(numpy_folder)
else:
  print('Numpy array folder exists')

In [ ]:
# Use for truncate HU value of 3DIRCAD dataset
def truncate_HU_value(range1, range2, img_path, save_path):
    print("*** Truncating HU value to eliminate superfluous information ***")
    for idx in range(range1, range2):
      # Due to the naming convention of 3DIRCAD dataset
        if idx < 10:
          img, img_header = load(img_path + '/ircad_e0' + str(idx) + '_orig.nii')
          img[img < -200] = -200
          img[img > 250] = 250
          img = np.array(img, dtype='int16')
          print('Saving image ' + str(idx))
          save(img, save_path + '/' + 'cad-volume-' + str(idx) + '.nii')
        else:
          img, img_header = load(img_path + '/ircad_e' + str(idx) + '_orig.nii')
          img[img < -200] = -200
          img[img > 250] = 250
          img = np.array(img, dtype='int16')
          print('Saving image ' + str(idx))
          save(img, save_path + '/' + 'cad-volume-' + str(idx) + '.nii')



# Remove tumor label of 3DIRCAD dataset
def remove_tumor_label(range1, range2, img_path, save_path):
    print("*** Removing tumor label ***")
    for idx in range(range1, range2):
        # Due to the naming convention of 3DIRCAD dataset
        if idx < 10:
          img, img_header = load(img_path + '/ircad_e0' + str(idx) + '_liver.nii')
          img[img == 2] = 1
          img = np.array(img, dtype='uint8')
          print('Saving image ' + str(idx))
          save(img, save_path + '/' + 'cad-segmentation-' + str(idx) + '.nii')
        else:
          img, img_header = load(img_path + '/ircad_e' + str(idx) + '_liver.nii')
          img[img == 2] = 1
          img = np.array(img, dtype='uint8')
          print('Saving image ' + str(idx))
          save(img, save_path + '/' + 'cad-segmentation-' + str(idx) + '.nii')



truncate_HU_value(range1=1, range2=21, img_path=testing_folder, save_path=processed_testing_folder)
remove_tumor_label(range1=1, range2=21, img_path=testing_folder, save_path=processed_testing_folder)

In [ ]:
def create_test_data():
    print('-' * 30)
    print('Creating test data...')
    print('-' * 30)

    imgs_test = []
    masks_test = []
    testing_images_files = []
    testing_masks_files = []
    
    # List out all of the item available in the folder
    item_list = os.listdir(testing_folder)
    np.sort(item_list)
    
    # Adjust the range based on which CT scan you want to test
    for idx in range(1,2):
      testing_images_files.append('cad-volume-' + str(idx) + '.nii')
      testing_masks_files.append('cad-segmentation-' + str(idx) + '.nii')

    # Load up the CT scan and transformed them into arrays
    for orig, liver in zip(testing_images_files, testing_masks_files):
        print('Processing: ' + orig)
        print('Processing: ' + liver)
        testing_image = nibabel.load(os.path.join(processed_testing_folder, orig))
        testing_mask = nibabel.load(os.path.join(processed_testing_folder, liver))
        
        print("Total testing slices before eliminate non-liver slice: " + str(testing_image.shape[2]))
        for k in range(testing_image.shape[2]):
            image_2d = np.array(testing_image.get_fdata()[::2, ::2, k])
            mask_2d = np.array(testing_mask.get_fdata()[::2, ::2, k])
            # If you want to test with uncropped data, comment out the line below
            if len(np.unique(mask_2d)) != 1:
              masks_test.append(mask_2d)
              imgs_test.append(image_2d)
        
    
    print("Total testing slices after eliminate non-liver slice: " + str(len(imgs_test)))

    # Transform orginal array (0,1,2) into array with format (2,0,1)
    imgst = np.ndarray((len(imgs_test), image_rows, image_cols), dtype=np.uint8)
    maskst = np.ndarray((len(masks_test), image_rows, image_cols), dtype=np.uint8)

    for index, img in enumerate(imgs_test):
        imgst[index, :, :] = img

    for index, mask in enumerate(masks_test):
        maskst[index, :, :] = mask

    np.save(numpy_folder + '/' + 'imgs_test.npy', imgst)
    np.save(numpy_folder + '/' + 'imgs_mask.npy', maskst)
    print('Saving to .npy files done.')
    

# Load up the numpy array
def load_test_data():
    print('--- Loading test images ---')
    imgs_test = np.load(numpy_folder + '/' + 'imgs_test.npy')
    masks_test = np.load(numpy_folder + '/' + 'imgs_mask.npy')
    return imgs_test, masks_test



if __name__ == '__main__':
    create_test_data()


In [5]:
"""
- Author DebeshJha
- Link https://github.com/DebeshJha/ResUNetplusplus_with-CRF-and-TTA
"""

# ResUNet++ model
def squeeze_excite_block(inputs, ratio=8):
    init = inputs
    channel_axis = -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    x = Multiply()([init, se])
    return x


def stem_block(x, n_filter, strides):
    x_init = x

    ## Conv 1
    x = Conv2D(n_filter, (3, 3), padding="same", strides=strides, kernel_regularizer=l2(weight_decay))(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(n_filter, (3, 3), padding="same", kernel_regularizer=l2(weight_decay))(x)

    ## Shortcut
    s = Conv2D(n_filter, (1, 1), padding="same", strides=strides, kernel_regularizer=l2(weight_decay))(x_init)
    s = BatchNormalization()(s)

    ## Add
    x = Add()([x, s])
    x = squeeze_excite_block(x)
    return x


def resnet_block(x, n_filter, strides=1):
    x_init = x

    ## Conv 1
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(n_filter, (3, 3), padding="same", strides=strides, kernel_regularizer=l2(weight_decay))(x)
    ## Conv 2
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(n_filter, (3, 3), padding="same", strides=1, kernel_regularizer=l2(weight_decay))(x)

    ## Shortcut
    s = Conv2D(n_filter, (1, 1), padding="same", strides=strides, kernel_regularizer=l2(weight_decay))(x_init)
    s = BatchNormalization()(s)

    ## Add
    x = Add()([x, s])
    x = squeeze_excite_block(x)
    return x


def aspp_block(x, num_filters, rate_scale=1):
    x1 = Conv2D(num_filters, (3, 3), dilation_rate=(6 * rate_scale, 6 * rate_scale), padding="SAME", kernel_regularizer=l2(weight_decay))(x)
    x1 = BatchNormalization()(x1)

    x2 = Conv2D(num_filters, (3, 3), dilation_rate=(12 * rate_scale, 12 * rate_scale), padding="SAME", kernel_regularizer=l2(weight_decay))(x)
    x2 = BatchNormalization()(x2)

    x3 = Conv2D(num_filters, (3, 3), dilation_rate=(18 * rate_scale, 18 * rate_scale), padding="SAME", kernel_regularizer=l2(weight_decay))(x)
    x3 = BatchNormalization()(x3)

    x4 = Conv2D(num_filters, (3, 3), padding="SAME", kernel_regularizer=l2(weight_decay))(x)
    x4 = BatchNormalization()(x4)

    y = Add()([x1, x2, x3, x4])
    y = Conv2D(num_filters, (1, 1), padding="SAME", kernel_regularizer=l2(weight_decay))(y)
    return y


def attention_block(g, x):
    # g: Output of Parallel Encoder block
    # x: Output of Previous Decoder block

    filters = x.shape[-1]

    g_conv = BatchNormalization()(g)
    g_conv = Activation("relu")(g_conv)
    g_conv = Conv2D(filters, (3, 3), padding="SAME", kernel_regularizer=l2(weight_decay))(g_conv)

    g_pool = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(g_conv)

    x_conv = BatchNormalization()(x)
    x_conv = Activation("relu")(x_conv)
    x_conv = Conv2D(filters, (3, 3), padding="SAME", kernel_regularizer=l2(weight_decay))(x_conv)

    gc_sum = Add()([g_pool, x_conv])

    gc_conv = BatchNormalization()(gc_sum)
    gc_conv = Activation("relu")(gc_conv)
    gc_conv = Conv2D(filters, (3, 3), padding="SAME", kernel_regularizer=l2(weight_decay))(gc_conv)

    gc_mul = Multiply()([gc_conv, x])
    return gc_mul


class ResUnetPlusPlus:
    def __init__(self, input_size=256):
        self.input_size = input_size

    def build_model(self):
        n_filters = [32, 64, 128, 256, 512]
        inputs = Input((self.input_size, self.input_size, 1))

        c0 = inputs
        c1 = stem_block(c0, n_filters[0], strides=1)

        ## Encoder
        c2 = resnet_block(c1, n_filters[1], strides=2)
        c3 = resnet_block(c2, n_filters[2], strides=2)
        c4 = resnet_block(c3, n_filters[3], strides=2)

        ## Bridge
        b1 = aspp_block(c4, n_filters[4])

        ## Decoder
        d1 = attention_block(c3, b1)
        d1 = UpSampling2D((2, 2))(d1)
        d1 = Concatenate()([d1, c3])
        d1 = resnet_block(d1, n_filters[3])

        d2 = attention_block(c2, d1)
        d2 = UpSampling2D((2, 2))(d2)
        d2 = Concatenate()([d2, c2])
        d2 = resnet_block(d2, n_filters[2])

        d3 = attention_block(c1, d2)
        d3 = UpSampling2D((2, 2))(d3)
        d3 = Concatenate()([d3, c1])
        d3 = resnet_block(d3, n_filters[1])

        ## output
        outputs = aspp_block(d3, n_filters[0])
        outputs = Conv2D(1, (1, 1), padding="same", kernel_regularizer=l2(weight_decay))(outputs)
        outputs = Activation("sigmoid")(outputs)

        ## Model
        model = Model(inputs, outputs)

        return model


# Preprocess test data by adding a new dimension in order to feed the data to network
def preprocess(imgs):
    print('--- Preprocessing images ---')
    imgs_p = np.ndarray((imgs.shape[0], image_rows, image_cols), dtype=np.uint8)
    for i in range(imgs.shape[0]):
        imgs_p[i] = resize(imgs[i], (image_rows, image_cols), preserve_range=True)

    imgs_p = imgs_p[..., np.newaxis]
    return imgs_p


def get_dice_coef(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    return (2. * intersection + smooth) / (np.sum(y_true) + np.sum(y_pred) + smooth)

def get_recall(y_true, y_pred):
    y_pred = y_pred > 0.5
    y_pred = y_pred.astype(np.int32)
    m = tf.keras.metrics.Recall()
    m.update_state(y_true, y_pred)
    r = m.result().numpy()
    m.reset_states()
    return r

def get_precision(y_true, y_pred):
    y_pred = y_pred > 0.5
    y_pred = y_pred.astype(np.int32)
    m = tf.keras.metrics.Precision()
    m.update_state(y_true, y_pred)
    r = m.result().numpy()
    m.reset_states()
    return r

def get_metrics(y_true, y_pred):
    y_pred = y_pred.flatten()
    y_true = y_true.flatten()

    dice_coef_val = get_dice_coef(y_true, y_pred)

    y_true = y_true.astype(np.int32)
   
    recall_value = get_recall(y_true, y_pred)
    precision_value = get_precision(y_true, y_pred)

    return [dice_coef_val,recall_value, precision_value]


def predict():
    for idx in range(1):
        print('-' * 30)
        print('Loading model and preprocessing test data...' + str(idx))
        print('-' * 30)

        arch = ResUnetPlusPlus(input_size=256)
        model = arch.build_model()
        model.load_weights(weight_path)

        optimizer = SGD(lr=1e-5, momentum=0.9, nesterov=True)
        metrics = [get_dice_coef, Recall(), Precision()]
        model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=metrics)

        #  load 3D data
        img_test = np.load('/content/drive/My Drive/3DIRCAD-Preprocessed-test/numpy/imgs_test.npy')

        img_test = preprocess(img_test)
        img_test = img_test.astype('float32')

        mean = np.mean(img_test)  # mean for data centering
        std = np.std(img_test)  # std for data normalization

        img_test -= mean
        img_test /= std

        
        #  load liver mask
        mask = np.load('/content/drive/My Drive/3DIRCAD-Preprocessed-test/numpy/imgs_mask.npy')
        mask = preprocess(mask)
        mask = mask.astype('float32')
        
        print('-' * 30)
        print('Predicting masks on test data...' + str(idx))
        print('-' * 30)

        imgs_mask_test_result = model.predict(img_test, verbose=1)

        #for k in range(len(imgs_mask_test_result)):
            #imgs_mask_test_result[k][:, :, :] = imgs_mask_test_result[k][:, ::-1, :]
        
        result = get_metrics(mask, imgs_mask_test_result)
    
        print(result)
        
        
if __name__ == '__main__':
    predict()

------------------------------
Loading model and preprocessing test data...0
------------------------------
--- Preprocessing images ---
--- Preprocessing images ---
------------------------------
Predicting masks on test data...0
------------------------------
4/4 [==============================] - 8s 2s/step
[0.9722369373861783, 0.9738993, 0.9891086]
